# Temperature data per country

In [ ]:
import numpy as np
import pickle
import pandas as pd
import os
from sklearn.preprocessing import scale

# Using CRU dataset

In [ ]:
c = pd.read_csv('utils/countries_wb.csv', dtype=str, delimiter=';', header=None)
countries = list(c[0])

In [ ]:
countries_monthly = {}

for country in countries:
    for file in os.listdir('utils/data/temp_country/'):
        if country in file:
            countries_monthly[country] = pd.read_csv('utils/data/temp_country/{}'.format(file), header=3, delimiter='    ', engine='python')

In [ ]:
# we do not have temperature data for these countries

not_included = []

for country in countries:
    if country not in list(countries_monthly.keys()):
        not_included.append(country)

In [ ]:
not_included

In [ ]:
for country, df in countries_monthly.items():
    df.columns = df.columns.str.replace(' ', '')
    #countries_monthly[country].drop(labels=['MAM', 'JJA', 'SON', 'DJF', 'ANN'], axis=1, inplace=True)
    months = df.loc[:, 'JAN':'DEC']
    countries_monthly[country]['AVG'] = months.mean(axis=1)

In [ ]:
# check
countries_monthly['Italy']

In [ ]:
# better save this precious data
t = open('utils/data/temp.pkl', 'wb')
pickle.dump(countries_monthly, t)
t.close()

## *old:* Data until 2016

Monthly temperature data for countries comes from the [World Bank](https://climateknowledgeportal.worldbank.org/download-data)

In [ ]:
temp = pd.read_csv('utils/data/temp.csv', sep=', ', usecols=range(4), dtype=object)
temp.tail()

In [ ]:
countries = []
c = temp.Country.unique()

for country in c:
    countries.append(country.strip())

In [ ]:
temp_dict = {}
t_dict = {}
t_dict_std = {}

for country in countries:
    temp_dict[country] = temp[temp.Country == country]
    temp_dict[country].drop(columns=['Statistics', 'Country'], inplace=True)
    t_dict[country] = temp_dict[country].astype(float).groupby('Year')['Temperature - (Celsius)'].mean()
    t_dict[country].rename('Temperature', inplace=True)
    t_dict_std[country] = pd.Series(index=t_dict[country].index, data=scale(list(t_dict[country])))
    t_dict_std[country].rename('Temperature', inplace=True)

In [ ]:
# check
t_dict_std['Albania']